In [1]:
{-# LANGUAGE OverloadedStrings #-}

import AdventOfCode
import qualified Data.Vector as V
import qualified Data.Attoparsec.ByteString.Char8 as A

input <- dayLines 8

In [2]:
import Data.Functor (($>))

data Instruction = Jmp | Acc | Nop deriving (Eq, Show)

parseInstr = do
    instr <- A.choice
        [ "jmp" $> Jmp
        , "acc" $> Acc
        , "nop" $> Nop
        ]
    A.skipSpace
    arg <- A.signed A.decimal
    pure (instr, arg)

instructions = V.fromList $ map (parsed parseInstr) input

In [3]:
data MachineState = MachineState
    { msPointer :: Int
    , msAccumulator :: Int
    , msInstructions :: V.Vector (Instruction, Int)
    } deriving (Eq, Show)

stepInstruction :: MachineState -> MachineState
stepInstruction ms = let
    pointer = msPointer ms
    currInstr = msInstructions ms V.! pointer
    ms' = case currInstr of
        (Nop, _) -> ms { msPointer = pointer + 1 }
        (Acc, v) -> ms
            { msPointer = pointer + 1
            , msAccumulator = msAccumulator ms + v
            }
        (Jmp, o) -> ms { msPointer = pointer + o}
    in ms'

In [4]:
import qualified Data.Set as S

findAccumulatorBeforeRepeat :: S.Set Int -> MachineState -> Int
findAccumulatorBeforeRepeat pointers ms = let
    currPointer = msPointer ms
    in if currPointer `S.member` pointers
        then msAccumulator ms
        else let
            pointers' = S.insert currPointer pointers
            ms' = stepInstruction ms
            in findAccumulatorBeforeRepeat pointers' ms'

In [5]:
findAccumulatorBeforeRepeat S.empty $ MachineState 0 0 instructions

2058

In [6]:
runSwappingInstructions :: S.Set Int -> Bool -> MachineState -> Either Int Int
runSwappingInstructions pointers changed ms
    | currPointer == V.length (msInstructions ms) = Right (msAccumulator ms)
    | currPointer `S.member` pointers = Left (msAccumulator ms)
    | fst currInstr `elem` [Jmp, Nop] && not changed = let
        pointers' = S.insert currPointer pointers
        ms' = stepInstruction ms
        ms'' = stepInstruction (swapInstruction ms)
        in runSwappingInstructions pointers' False ms' <> runSwappingInstructions pointers' True ms''
    | otherwise = let
        pointers' = S.insert currPointer pointers
        ms' = stepInstruction ms
        in runSwappingInstructions pointers' changed ms'
    where
        currPointer = msPointer ms
        currInstr = msInstructions ms V.! currPointer
        swapInstruction ms = let
            (op, arg) = currInstr
            newInstr = case op of
                Jmp -> (Nop, arg)
                Nop -> (Jmp, arg)
                _ -> error "you made a mistake somewhere"
            ms' = ms { msInstructions = msInstructions ms V.// [(currPointer, newInstr)]}
            in ms'

In [7]:
runSwappingInstructions S.empty False $ MachineState 0 0 instructions

Right 1000